In [1]:
from functools import partial 

from sepal_ui import widgetFactory as wf
from sepal_ui import widgetBinding as wb
from sepal_ui.scripts import utils as su
import ipyvuetify as v

from scripts import gee_process as gee_run 
from utils import messages as ms
from utils import parameters as pm

In [2]:
#class io as mutable object 
class Ga_gee_io:
    def __init__(self):
        #input 
        self.year = 2020
        self.range = ['2020-01-01', '2020-12-31']
        
ga_gee_io = Ga_gee_io()

In [3]:
#output 
ga_gee_output = wf.OutputWidget(ms.GEE_INTRO)

In [4]:
from datetime import datetime

def print_value(data):
    print(data.value)

#custom date slider
ga_range_slider = v.RangeSlider(
    label='Select date range',
    class_="mt-6", 
    thumb_label='always', 
    v_model=[1,365],
    min=1,
    max=366,
    v_slots=[{
        'name': 'thumb_label',
        'variable': 'slider',
        'children' : 'slider.value'
    }]
)
def edit_display(widget, event, data, output):
    
    year = getattr(ga_gee_io, 'year')
    
    start = widget.v_model[0]
    start = '{0}{1:03}'.format(str(year%100), start)
    start = datetime.strptime(start, '%y%j')
    start = start.strftime("%Y-%m-%d")
    
    end = widget.v_model[1]
    end = '{0}{1:03}'.format(str(year%100), end)
    end = datetime.strptime(end, '%y%j')
    end = end.strftime("%Y-%m-%d")
    
    setattr(ga_gee_io, 'range', [start, end])
    su.displayIO(output, 'Value Selected: {}'.format(ga_gee_io.range))
    
    return    
    
ga_range_slider.on_event('change', partial(
    edit_display,
    output=ga_gee_output
))

In [5]:
#year selector 
ga_year = v.Select(items=pm.getAvailableYear(), label=ms.GEE_SELECT, v_model=2020)



#custom binding 
def reset_range(slider, output):
    year = getattr(ga_gee_io, 'year')
    
    slider.v_model = [1,365]
    
    start = '{0}{1:03}'.format(str(year%100), 1)
    start = datetime.strptime(start, '%y%j')
    start = start.strftime("%Y-%m-%d")
    
    end = '{0}{1:03}'.format(str(year%100), 365)
    end = datetime.strptime(end, '%y%j')
    end = end.strftime("%Y-%m-%d")
    
    setattr(ga_gee_io, 'range', [start, end])
    su.displayIO(output, 'Value Selected: {}'.format(ga_gee_io.range))
    
    return

def change_year(widget, event, data, output, slider):
    
    setattr(ga_gee_io, 'year', widget.v_model)
    su.displayIO(output, 'Value Selected: {}'.format(ga_gee_io.year))
    
    reset_range(slider, output)
    
    return 

ga_year.on_event('change', partial(
    change_year,
    slider=ga_range_slider,
    output=ga_gee_output
))

In [6]:
#btn 
ga_gee_btn = wf.ProcessBtn(ms.GEE_BTN)

#tile
id_ = 'gee_widget'
title = 'Retrieve alerts'

ga_gee = wf.Tile(
    id_,
    title,
    btn=ga_gee_btn,
    output=ga_gee_output,
    inputs=[ga_year, ga_range_slider]
)

In [7]:
#create a custom function for the process
def process_start(widget, event, data, output):
    
    asset = getattr(ga_aoi_io, 'assetId')
    year = getattr(ga_gee_io, 'year')
    date_range = getattr(ga_gee_io, 'range')
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    su.displayIO(output, asset)
    
    #check inputs
    if not wb.checkInput(asset, output, ms.NO_AOI): return su.toggleLoading(widget)
    if not wb.checkInput(year, output, ms.NO_YEAR): return su.toggleLoading(widget)
    if not wb.checkInput(date_range, output, ms.NO_YEAR): return su.toggleLoading(widget)

    try:
        gee_run.gee_process(asset, date_range, year, output) 
    except Exception as e: 
        su.displayIO(output, str(e), 'error')
    
    #toggle the loading button
    su.toggleLoading(widget)
    
    return 

ga_gee_btn.on_event('click', partial(
    process_start,
    output=ga_gee_output,
))
    

In [8]:
ga_gee

Layout(align_center=True, children=[Card(children=[Html(children=['Retrieve alerts'], tag='h2'), Flex(children…